In [65]:
from sklearn                 import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble        import RandomForestRegressor
from sklearn.linear_model    import LinearRegression
from sklearn.linear_model    import LogisticRegression
from sklearn                 import preprocessing
from sklearn.metrics         import r2_score
from pandas.core.frame       import DataFrame
from sklearn.linear_model    import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network  import MLPRegressor
from itertools               import permutations
from itertools               import combinations

import numpy                 as np
import matplotlib.pyplot     as plt
import pandas

In [57]:
# 1. Список регрессоров
#     а) LinearRegression
#     б) RidgeRegerssion
#     в) Lasso
#     г) ElasticNet
#     в) RandomForestRegressor 
#     д) Least Angle Regression(LARS)
#     е) LassoLars
#     ж) OrtogonalMatchingPursuit(OMP)
#     з) Bayesian Regression
#     и) LogisticRegression
#     к) RANSAC
#     л) Theil Sen
#     м) HuberRegressor
#     н) MLPRegressor
    
# 2. Для каждого подобрать оптимальные параметры и признаки
#     Отбор признаков:
#     Их не так много, можно руками.
#     Отбор параметров:
#     а) GridSearch
#     б) RandomizedSearch
# 3. Обучить выбороку на всех моделях с лучшими параметрами
# 4. Протестировать 
# 5. Выбрать лучшие модели

In [14]:
columns = ['face_x', 'face_y','face_width','face_height','face_cam_x', 'face_cam_y','face_cam_z',
          'torso_x','torso_y','torso_width','torso_height','torso_cam_x','torso_cam_y','torso_cam_z',
          'person_x','person_y','person_width','person_height','person_cam_x','person_cam_y','person_cam_z',
        'mean_cam_z']
df = pandas.read_csv('data.csv')
df_test = pandas.read_csv('test_data.csv')

face_X = df[['face_x',  'face_y', 'face_width', 'face_height']].values
torso_X = df[['torso_x',  'torso_y',  'torso_width',  'torso_height']].values
person_X = df[['person_x', 'person_y', 'person_width', 'person_height']].values

face_y = df['face_cam_z'].values
torso_y = df['torso_cam_z'].values
person_y = df['person_cam_z'].values

face_X_test = df_test[['face_x',  'face_y', 'face_width', 'face_height']].values
torso_X_test = df_test[['torso_x',  'torso_y',  'torso_width',  'torso_height']].values
person_X_test = df_test[['person_x', 'person_y', 'person_width', 'person_height']].values

face_y_test = df_test['face_cam_z'].values
torso_y_test = df_test['torso_cam_z'].values
person_y_test = df_test['person_cam_z'].values

X = [face_X, torso_X, person_X]
Y = [face_y, torso_y, person_y]

X_test = [face_X_test, torso_X_test, person_X_test]
Y_test = [face_y_test, torso_y_test, person_y_test]

for i, (x, y) in enumerate(zip(X, Y)):
    keep = ~np.isnan(y)    
    X[i] = x[keep,:]
    Y[i] = y[keep]
    print(X[i].shape, Y[i].shape)
    
for i, (x, y) in enumerate(zip(X_test, Y_test)):
    keep = ~np.isnan(y)    
    X_test[i] = x[keep,:]
    Y_test[i] = y[keep]
    print(X_test[i].shape, Y_test[i].shape)
    
# remove long data
for i, (x, y) in enumerate(zip(X, Y)):
    print(y.shape)
    short_data = y < 3.
    Y[i] = y[short_data]
    X[i] = x[short_data, :]

((265, 4), (265,))
((209, 4), (209,))
((207, 4), (207,))
((15, 4), (15,))
((1, 4), (1,))
((0, 4), (0,))
(265,)
(209,)
(207,)


In [64]:
numbersList = list(range(4))
indeces = list(combinations(numbersList,2)) + list(combinations(numbersList,3)) + list(combinations(numbersList,4))

def futuresDroper(rgr,params,X,y,kf,n_iter=0):
    if n_iter>0:
        grid = RandomizedSearchCV(cv=kf,estimator=rgr,n_jobs=-1,scoring='r2',n_iter=n_iter,param_distributions=params)
    elif n_iter == 0:
        grid = GridSearchCV(cv=kf,estimator=rgr,n_jobs=-1,scoring='r2',param_grid=params)        
    else: 
        return 
        
    best = dict(futures = (),
           params = dict(),
           score = -500.,
           estimator = '')

    for i in indeces:
        grid.fit(X=X[:,i],y=y)
        if abs(1. - best['score']) > abs(1. - grid.best_score_):
            (best['futures'], best['params'], best['score'],best['estimator']) = (i,grid.best_params_,grid.best_score_,grid.best_estimator_)
    
    return best

In [61]:
rgr = RandomForestRegressor(n_estimators=100)
kf = KFold(n_splits=10,shuffle=True)

params = dict(
                max_depth = range(1,31),
                min_samples_split = range(2,11),
                min_samples_leaf = range(1,11),
                min_weight_fraction_leaf = np.linspace(0,0.5,10),
#                 max_features = range(1,5),
                max_leaf_nodes = range(2,11),
                min_impurity_decrease = np.linspace(0,10,20),
                verbose = range(11)
                )
# ~2e10 combinations :)
grid = RandomizedSearchCV(cv=kf,estimator=rgr,n_jobs=-1,scoring='r2',n_iter=500,param_distributions=params)
# grid = GridSearchCV(cv=kf,estimator=rgr,n_jobs=-1,scoring='r2',param_grid=params)

In [62]:
#It takes really long time
for i in range(3):
    d = futuresDroper(rgr,params,X[i],Y[i],kf,n_iter=500)
    print('::::::::::',i)
    print(d)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


...

/home/konstantin/.local/lib/python2.7/site-packages/sklearn/model_selection/_search.py:842: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
/home/konstantin/.local/lib/python2.7/site-packages/sklearn/model_selection/_search.py:842: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

/home/konstantin/.local/lib/python2.7/site-packages/sklearn/model_selection/_search.py:842: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
# Face
# {'estimator': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
#            max_features='auto', max_leaf_nodes=8,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=7, min_samples_split=3,
#            min_weight_fraction_leaf=0.05555555555555555, n_estimators=100,
#            n_jobs=None, oob_score=False, random_state=None, verbose=8,
#            warm_start=False), 
#  'params': {'min_impurity_decrease': 0.0, 
#             'verbose': 8, 
#             'max_leaf_nodes': 8, 
#             'min_samples_leaf': 7, 
#             'min_weight_fraction_leaf': 0.05555555555555555, 
#             'min_samples_split': 3, 
#             'max_depth': 10}, 
#  'score': 0.7830240411591205, 
#  'futures': (1, 2, 3)}

# Torso
# {'estimator': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
#            max_features='auto', max_leaf_nodes=10,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=1, min_samples_split=10,
#            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
#            oob_score=False, random_state=None, verbose=0, warm_start=False), 
#  'params': {'min_impurity_decrease': 0.0, 
#             'verbose': 0, 
#             'max_leaf_nodes': 10, 
#             'min_samples_leaf': 1, 
#             'min_weight_fraction_leaf': 0.0, 
#             'min_samples_split': 10, 
#             'max_depth': 19}, 
#  'score': 0.6159092780433589, 
#  'futures': (2, 3)}

# Person
# {'estimator': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=29,
#            max_features='auto', max_leaf_nodes=9,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=2, min_samples_split=10,
#            min_weight_fraction_leaf=0.05555555555555555, n_estimators=100,
#            n_jobs=None, oob_score=False, random_state=None, verbose=6,
#            warm_start=False), 
#  'params': {'min_impurity_decrease': 0.0, 
#             'verbose': 6, 
#             'max_leaf_nodes': 9, 
#             'min_samples_leaf': 2, 
#             'min_weight_fraction_leaf': 0.05555555555555555, 
#             'min_samples_split': 10, 
#             'max_depth': 29}, 
#  'score': 0.5875569151775498, 
#  'futures': (2, 3)}

In [34]:
grid.fit(X=X[2],y=Y[2])
grid.best_params_,grid.best_score_

/home/konstantin/.local/lib/python2.7/site-packages/sklearn/model_selection/_search.py:842: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


({'max_depth': 17,
  'max_features': 3,
  'max_leaf_nodes': 7,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 2,
  'min_samples_split': 9,
  'min_weight_fraction_leaf': 0.05555555555555555,
  'verbose': 6},
 0.5282769213881424)

In [119]:
rgr = LinearRegression(copy_X=True)
kf = KFold(n_splits=10,shuffle=True)

params = dict(
                fit_intercept = [True,False],
                normalize = [True,False]
              )

# grid = RandomizedSearchCV(cv=kf,estimator=rgr,n_jobs=-1,scoring='r2',n_iter=500,param_distributions=params)
grid = GridSearchCV(cv=kf,estimator=rgr,n_jobs=-1,scoring='r2',param_grid=params)

In [120]:
grid.fit(X=X[2],y=Y[2])
print (grid.best_params_,grid.best_score_)

({'normalize': True, 'fit_intercept': True}, 0.5591856931322622)


/home/konstantin/.local/lib/python2.7/site-packages/sklearn/model_selection/_search.py:842: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [26]:
rgr = LogisticRegression(penalty='l2')
kf = KFold(n_splits=10,shuffle=True)

params = dict(
                solver = ['newton-cg','sag','lbfgs'],
                tol = np.linspace(1e-4,1,20),
                C = range(1,1001),
                fit_intercept = [True,False],
                max_iter = range(100,301)
              )

grid = RandomizedSearchCV(cv=kf,estimator=rgr,n_jobs=-1,scoring='r2',n_iter=500,param_distributions=params)
# grid = GridSearchCV(cv=kf,estimator=rgr,n_jobs=-1,scoring='r2',param_grid=params)

In [27]:
# grid.fit(X=preprocessing.scale(X[2]),y=np.round(1000*Y[2]))
# print (grid.best_params_,grid.best_score_)
grid.fit(X=X[2][:,2:] / np.array([620, 480]),y=np.round(10*Y[2]))
print (grid.best_params_,grid.best_score_)

({'fit_intercept': True, 'C': 343, 'max_iter': 258, 'tol': 0.15797894736842105, 'solver': 'lbfgs'}, 0.3157545893862177)


/home/konstantin/.local/lib/python2.7/site-packages/sklearn/model_selection/_search.py:842: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [44]:
rgr = MLPRegressor()
kf = KFold(n_splits=10,shuffle=True)

params = dict(
                hidden_layer_sizes = [(100,),(100,50),(100,100)],
                activation = ['identity','logistic','tanh', 'relu'],
                alpha = np.linspace(1e-4,10,30),
                solver = ['sgd','lbfgs'],
                max_iter = range(100,500,50),
                tol = np.linspace(1e-4,1,20),
                
              )

grid = RandomizedSearchCV(cv=kf,estimator=rgr,n_jobs=-1,scoring='r2',n_iter=500,param_distributions=params)
# grid = GridSearchCV(cv=kf,estimator=rgr,n_jobs=-1,scoring='r2',param_grid=params)

In [45]:
# grid.fit(X=preprocessing.scale(X[2]),y=Y[2])
# print (grid.best_params_,grid.best_score_)
grid.fit(X=X[0] / np.array([620, 480]),y=Y[0])
print (grid.best_params_,grid.best_score_)

({'hidden_layer_sizes': (100, 100), 'activation': 'relu', 'max_iter': 100, 'tol': 0.0001, 'alpha': 0.689748275862069, 'solver': 'lbfgs'}, 0.7436736508044619)


In [48]:
rgr = Lasso(normalize=True,copy_X=True)
kf = KFold(n_splits=10,shuffle=True)

params = dict(
                fit_intercept = [True,False],
                alpha = np.linspace(1e-4,10,30),
                max_iter = range(100,500,50),
                positive = [True,False]
              )

# grid = RandomizedSearchCV(cv=kf,estimator=rgr,n_jobs=-1,scoring='r2',n_iter=500,param_distributions=params)
grid = GridSearchCV(cv=kf,estimator=rgr,n_jobs=-1,scoring='r2',param_grid=params)

In [51]:
# grid.fit(X=preprocessing.scale(X[2]),y=Y[2])
# print (grid.best_params_,grid.best_score_)
grid.fit(X=X[2],y=Y[2])
print (grid.best_params_,grid.best_score_)

({'alpha': 0.0001, 'max_iter': 450, 'positive': False, 'fit_intercept': True}, 0.5257748198524566)


/home/konstantin/.local/lib/python2.7/site-packages/sklearn/model_selection/_search.py:842: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
